In [ ]:
val reportFolder = "reports"
val executionRounds = 10

In [ ]:
import java.io.File
import java.util.Properties

val properties = Properties()
File("gradle.properties").bufferedReader().use {
    properties.load(it)
}

val executedTask = properties.getProperty("scenario.task") ?: "assemble"

val projectPath = properties.getProperty("project.path") ?: properties.getProperty("project.git.directory")

val projectName: String = projectPath?.substringAfterLast("\\")?.substringAfterLast("/")?.removeSuffix(".git") ?: "User"

In [ ]:
enum class CompilerMetrics(val readableName: String) {
    GRADLE_TASK("Total Gradle task time"),

    COMPILATION_ROUND("Sources compilation round"),
    
    //compiler metrics
    COMPILER_INITIALIZATION("Compiler initialization time"),
    CODE_ANALYSIS("Compiler code analysis"),
    IR_TRANSLATION("Compiler IR translation"),
    IR_LOWERING("Compiler IR lowering"),
    IR_GENERATION("Compiler IR generation"),
    ;
}

In [ ]:
%use dataframe

In [ ]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

//Json report print time in milliseconds 
fun readJsonReportFile(file: File): Map<CompilerMetrics, Long?>? {
    if (file.extension != "json" || file.readText().isBlank())
        return null
    val report = DataFrame.read(file)

    if ((report.get("startParameters").get("tasks")[0] as List<String>).get(0) != executedTask)
        return null

    val timeMetrics = report.get("aggregatedMetrics").get("buildTimes").get("buildTimesNs")[0] as DataRow<*>

    return CompilerMetrics.values().associate {
        val value = timeMetrics.getOrNull(it.name)?.let {
            when (it) {
                is Int -> it.toLong()
                is Long -> it
                is String -> it.toLong()
                else -> throw IllegalStateException("Unknown type ${it.javaClass} to convert to Long")
            }
        }
        Pair(it, value)
    }
}

In [ ]:
import kotlin.streams.toList

//Text report print time in seconds with 2 decemal digits, so there is an error during conversion to nanoSeconds
fun readTextReportFile(file: File): Map<CompilerMetrics, Long?>? {
    val isValidReportFile = file.extension == "txt" && file.readText().isNotEmpty()  && file.readText().contains("tasks = [$executedTask]")
    if (!isValidReportFile) return null
    
    val aggregatedTimeMetrics = file.bufferedReader().use {
        it.lines().dropWhile { !it.startsWith("Time metrics:") }.takeWhile { it.isNotBlank() }.toList()
    }

    return CompilerMetrics.values().associate { metric ->
        val metricLine = aggregatedTimeMetrics.firstOrNull { it.trim().startsWith(metric.readableName) }
        val value = metricLine?.substringAfter(": ")?.substringBefore(" ")?.replace(",", "")?.toDouble()
        Pair(metric, value?.times(1_000_000_000)?.toLong())
    }
}


# Read compiler metrics


In [ ]:
%use kandy

In [ ]:
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import java.nio.file.Files
import kotlin.io.path.Path

fun readMetrics(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?
) = directory.listFiles()
    .sortedByDescending(File::lastModified)
    .mapNotNull { readReportFile(it) }
    .take(executionRounds)
    .toList()

fun collectMeanValue(
    metrics: List<Map<CompilerMetrics, Long?>>,
): Map<String, Any?> {
    val rows = CompilerMetrics.values().map { compilerMetric ->
        val values = metrics.map { it[compilerMetric] }.filterNotNull()
        Pair(compilerMetric.name, values)
    }.filter { 
        val allValuesPresent = (it.second.size == executionRounds)
        if (!allValuesPresent) {
            println("Unable to calculate mean value for ${it.first}, only ${it.second.size} runs are found")
        }
        allValuesPresent
    }.toTypedArray()

    val meanValue = dataFrameOf(*rows).mean()

    return CompilerMetrics.values().associate {
        Pair(it.name, meanValue.getOrNull(it.name))
    }
} 


# Collect data into dataframe

In [ ]:
val kotlinVersionColumn = "kotlinVersion"
val buildScenarioColumn = "buildScenario"

val columnNames = CompilerMetrics.values().map { it.name }.toMutableList()
columnNames.add(kotlinVersionColumn)
columnNames.add(buildScenarioColumn)

fun collectResultsIntoDataFrame(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?,
): AnyFrame {
    val rows = directory.listFiles().filter { it.isDirectory() }.flatMap { versionDir ->
        versionDir.listFiles().filter { it.isDirectory() }.map { scenarioDir ->
            val buildMetrics = readMetrics(scenarioDir, readReportFile)
            //rename scenarios for graphics only
            val scenario = when (scenarioDir.name) {
                "clean_build" -> "Clean build"
                "incremental_abi_build" -> "Incremental build for ABI changes"
                "incremental_non_abi_build" -> "Incremental build for non-ABI changes"
                else -> scenarioDir.name
            }

            println("\n Validate \'$scenario\' scenario for ${versionDir.name} kotlin version")
            val row = collectMeanValue(buildMetrics).toMutableMap()
            row.put(kotlinVersionColumn, versionDir.name)
            row.put(buildScenarioColumn, scenario)
            row
        }
    }

    val db = columnNames.map { compilerMetric -> Pair(compilerMetric, rows.map { it[compilerMetric] })}.toTypedArray()
    return dataFrameOf(*db)
}

In [ ]:
import java.util.concurrent.TimeUnit
import kotlin.time.DurationUnit
import kotlin.time.toDuration

enum class TimeTransformation(val printTime: String, val transformation: (Double?) -> Double?) {
    NANOSECONDS("nanosaconds", { it }),
    MILLISECONDS("milliseconds", { 
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.MILLISECONDS) 
    }),
    SECONDS("seconds", {
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.SECONDS) 
    })
    ;
}

fun getTimeTranformationForColumn(df: DataFrame<Any?>, column: String): TimeTransformation {
    val meanValue = df.mean(column, skipNA = true)
    val ratio = 1_000
    return when {
        meanValue < 10 * ratio -> TimeTransformation.NANOSECONDS
        meanValue < 10 * ratio * ratio -> TimeTransformation.MILLISECONDS
        else -> TimeTransformation.SECONDS
    }
}

In [ ]:
var df = collectResultsIntoDataFrame(File(reportFolder), ::readTextReportFile)
    .sortBy(kotlinVersionColumn)
val timeTransformation = getTimeTranformationForColumn(df, CompilerMetrics.COMPILATION_ROUND.name)

 CompilerMetrics.values().forEach { 
     df = df.update(it.name).with { timeTransformation.transformation(it as Double) }
 }

val dataFrame = df
val scenarios = dataFrame[buildScenarioColumn]

In [ ]:
val plots = scenarios.distinct().toList().map { scenario ->
    dataFrame.filter { buildScenario == scenario }.plot {
        layout {
            title = "$projectName project"
            subtitle = "$scenario"
            style { 
                legend {
                    position = LegendPosition.Top
                }
            }
        }
        bars {
            x(kotlinVersionColumn) {
                axis.name = "Gradle compilation time"
            }
            y(CompilerMetrics.COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
        }
        
    }
}
plotGrid(plots, 1)

In [ ]:
val incrementalScenatiosLabels = listOf(Pair("Incremental build for ABI changes", "ABI changes"), Pair("Incremental build for non-ABI changes", "non-ABI changes"))

In [ ]:
fun gradleBuildTime(scenarios: List<String>, name: String, scenarioBreaksLabeled: List<Pair<String, String>>? = null) =
    dataFrame.filter { buildScenario in scenarios }
        .plot {
            layout {
                title = "$projectName project - $name"
                style {
                    legend {
                        position = LegendPosition.Top
                    }
                }
            }
            bars {
                x(buildScenarioColumn) {
                    axis.name = "Gradle build time"
                    scenarioBreaksLabeled?.let {
                        axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                    }
                }
                y(GRADLE_TASK.name) {
                    axis.name = "Time (${timeTransformation.printTime})"
                }
                fillColor(kotlinVersionColumn) {
                    legend.name = ""
                }
            }
        }

In [ ]:
gradleBuildTime(listOf("Clean build"), "Clean build scenario")

In [ ]:
gradleBuildTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)


In [ ]:
fun kotlinCompilationTime(
    scenarios: List<String>,
    name: String,
    scenarioBreaksLabeled: List<Pair<String, String>>? = null
) = dataFrame.filter { buildScenario in scenarios }
    .plot {
        layout {
            title = "$projectName project - $name"
            style {
                legend {
                    position = LegendPosition.Top
                }
            }
        }
        bars {
            x(buildScenarioColumn) {
                axis.name = "Kotlin compilation time"
                scenarioBreaksLabeled?.let {
                    axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                }
            }
            y(COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
            fillColor(kotlinVersionColumn) {
                legend.name = ""
            }
        }
    }
    


In [ ]:
kotlinCompilationTime(listOf("Clean build"), "Clean build scenario")


In [ ]:
kotlinCompilationTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)

# Compare Gradle and Compiler time

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalTransform

fun getPlotForCompilerMetrics(
    scenario: String,
    graphicsCaption: String,
    vararg metrics: Pair<CompilerMetrics, String>
) = dataFrame
    .gather(*metrics.map { it.first.name }.toTypedArray()).into("metric", "time_ns")
    .filter { it["buildScenario"] == scenario }
    .plot {
        layout {
            title = "$projectName project"
            subtitle = scenario
            caption = graphicsCaption
            style {
                legend {
                    position = LegendPosition.Top
                }
            }
            size = Pair(800, 300)
        }
        barsH {
            y("kotlinVersion"<String>()) {
                axis.name = "Kotlin version"
            }
            x("time_ns"<Double>()) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
            fillColor("metric"<String>()) {
                legend {
                    name = ""
                    breaksLabeled(*metrics.map { Pair(it.first.name, it.second) }.toTypedArray())
                }
            }
            position = Position.stack()

        }
    }

In [ ]:
val versionByScenario =
    scenarios.distinct().toList().associate { name -> Pair(name as String, getPlotForCompilerMetrics(name, "Gradle compilation breakdown", CompilerMetrics.COMPILATION_ROUND to "Kotlin Compiler time", 
        CompilerMetrics.GRADLE_TASK to "Gradle time")) }

In [ ]:
versionByScenario.get("Clean build")

In [ ]:
versionByScenario.get("Incremental build for non-ABI changes")

In [ ]:
versionByScenario.get("Incremental build for ABI changes")

# Collect Compiler metrics

In [ ]:
val compilerMetrics = scenarios.distinct().toList().associate {
    Pair(
        it as String, getPlotForCompilerMetrics(
            it, "Kotlin compilation breakdown",
            CompilerMetrics.COMPILER_INITIALIZATION to "Initialization phase",
            CompilerMetrics.CODE_ANALYSIS to "Analysis phase",
            CompilerMetrics.IR_TRANSLATION to "Translation phase",
            CompilerMetrics.IR_LOWERING to "Lowering",
            CompilerMetrics.IR_GENERATION to "Generation"
        )
    )
}

In [ ]:
compilerMetrics.get("Clean build") 


In [ ]:
compilerMetrics.get("Incremental build for non-ABI changes") 

In [ ]:
compilerMetrics.get("Incremental build for ABI changes") 